#### Reference:-
https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html

# Importing the required libraries

In [1]:
import numpy as np # to manipulate the data
import matplotlib.pyplot as plt # to visualise the result of our model

import tensorflow
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input #to create the Embedding matrix
from tensorflow.keras.layers import LSTM # predicting using the Long Short Term Memory model
from tensorflow.keras.layers import Dropout #for the regularization(handling the overfitting)
from tensorflow.keras.layers import Dense #to create the output layer

# Configuration for the Project

In [2]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'deu-eng/deu.txt'

# Data Preparation

In [3]:
#Vectorizing the data
input_texts = [] # creating empty list to add the input sentences
target_texts = [] # creating empty list to add the target sentences
input_characters = set() # creating empty set to add the input language charecters(since we need only the unique charecter so we use set)
target_characters = set() # creating empty set to add the target language charecters(since we need only the unique charecter so we use set)
with open(data_path, 'r' , encoding = 'utf-8') as f: # reading the file from the directory
    lines = f.read().split('\n') # reading each line of the code
# we need to separate the english and german so we use for loop
for line in lines[: min(num_samples, len(lines) - 1)]: # we are going to take only 10000 sentences not more than that
    input_text, target_text, _ = line.split("\t") #since the english statement and german is separated by tab and remaining sentence also by tab so we are spliting with tab
    # We use "tab" as the "start sequence" character for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text) # appending the input text to the to the list
    target_texts.append(target_text) # appending the target text to the list
    for char in input_text:
        if char not in input_characters: # taking only the unique charecter of the input language(english)
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters: # taking only the unique charecter of the target language(german)
            target_characters.add(char)

In [4]:
# converting the set into list and sorting the charecters
input_characters = sorted(list(input_characters)) 
target_characters = sorted(list(target_characters))
# calculating the length of total charecter in the input and target language
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
#calculating the length of the longest sentence in both input and the output language
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [5]:
print("Number of samples:", len(input_texts)) # total sample of text that we are taking under consideration
print("Number of unique input tokens:", num_encoder_tokens) # length of charecters in input language
print("Number of unique output tokens:", num_decoder_tokens) # length of charecters in target language
print("Max sequence length for inputs:", max_encoder_seq_length) # words in the longest sentence in input language
print("Max sequence length for outputs:", max_decoder_seq_length) # words in the longest sentence in target language

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 85
Max sequence length for inputs: 15
Max sequence length for outputs: 45


In [6]:
# indexing the charecter like assigning 0 to first charecter and 1 to second and so on
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)]) 
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [7]:
input_token_index

{' ': 0,
 '!': 1,
 '"': 2,
 '$': 3,
 '%': 4,
 "'": 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '0': 9,
 '1': 10,
 '2': 11,
 '3': 12,
 '4': 13,
 '5': 14,
 '6': 15,
 '7': 16,
 '8': 17,
 '9': 18,
 ':': 19,
 '?': 20,
 'A': 21,
 'B': 22,
 'C': 23,
 'D': 24,
 'E': 25,
 'F': 26,
 'G': 27,
 'H': 28,
 'I': 29,
 'J': 30,
 'K': 31,
 'L': 32,
 'M': 33,
 'N': 34,
 'O': 35,
 'P': 36,
 'Q': 37,
 'R': 38,
 'S': 39,
 'T': 40,
 'U': 41,
 'V': 42,
 'W': 43,
 'Y': 44,
 'a': 45,
 'b': 46,
 'c': 47,
 'd': 48,
 'e': 49,
 'f': 50,
 'g': 51,
 'h': 52,
 'i': 53,
 'j': 54,
 'k': 55,
 'l': 56,
 'm': 57,
 'n': 58,
 'o': 59,
 'p': 60,
 'q': 61,
 'r': 62,
 's': 63,
 't': 64,
 'u': 65,
 'v': 66,
 'w': 67,
 'x': 68,
 'y': 69,
 'z': 70}

In [8]:
# creating the required input and the output to the encoder and decoder. Initiating all the arrays with zero
#len(input_texts) = total no sentences in the input language
#max_encoder_seq_length = longest sentence in the input language
#num_encoder_tokens = number of charecter in the input language

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

In [9]:
#One hot representation
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)): # 'i' will give the count and input_text = input_texts[i] and target_text = target_texts[i](extracting the sentence one by one)
    for t, char in enumerate(input_text): # 't' is the count and char will get sentence in the input_text[0](extracting the charecter one by one)
        encoder_input_data[i, t, input_token_index[char]] = 1.0 # assigning 1 to the charecter
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0 # after each word where ever there is space(' ') then also assigning 1
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

Here Decoder_target_data will result first from the context vector provided by the encoder and the same output will go to the input of the next decoder so we are adding the offset as shown above

# Model Creation

In [10]:
# Defining an input sequence and processing it.
encoder_inputs = tensorflow.keras.Input(shape=(None, num_encoder_tokens))
encoder = tensorflow.keras.layers.LSTM(latent_dim, return_state=True) # since we don't need the output  so return_state = True
encoder_outputs, state_h, state_c = encoder(encoder_inputs) # adding all the outcomes of encoder in the new variable

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = tensorflow.keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = tensorflow.keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True) #defining the LSTM for the decoder
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states) # keeping the output of the decoder
decoder_dense = tensorflow.keras.layers.Dense(num_decoder_tokens, activation="softmax") #dense layer to get the combined output sentence
decoder_outputs = decoder_dense(decoder_outputs) # passing all the out of the decoder to get the complete sentence

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = tensorflow.keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Training the Model

In [11]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("s2s", overwrite=True)

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 7s 888us/sample - loss: 1.4710 - accuracy: 0.6477 - val_loss: 1.3291 - val_accuracy: 0.6251
Epoch 2/100
8000/8000 [==============================] - 1s 172us/sample - loss: 1.0502 - accuracy: 0.7223 - val_loss: 1.0261 - val_accuracy: 0.7256
Epoch 3/100
8000/8000 [==============================] - 1s 170us/sample - loss: 0.8488 - accuracy: 0.7702 - val_loss: 0.8851 - val_accuracy: 0.7558
Epoch 4/100
8000/8000 [==============================] - 1s 170us/sample - loss: 0.7394 - accuracy: 0.7910 - val_loss: 0.8072 - val_accuracy: 0.7733
Epoch 5/100
8000/8000 [==============================] - 1s 170us/sample - loss: 0.6747 - accuracy: 0.8062 - val_loss: 0.7656 - val_accuracy: 0.7840
Epoch 6/100
8000/8000 [==============================] - 1s 170us/sample - loss: 0.6271 - accuracy: 0.8192 - val_loss: 0.7187 - val_accuracy: 0.7950
Epoch 7/100
8000/8000 [==============================] - 1

8000/8000 [==============================] - 1s 182us/sample - loss: 0.1353 - accuracy: 0.9607 - val_loss: 0.6917 - val_accuracy: 0.8474
Epoch 56/100
8000/8000 [==============================] - 1s 180us/sample - loss: 0.1329 - accuracy: 0.9611 - val_loss: 0.7069 - val_accuracy: 0.8454
Epoch 57/100
8000/8000 [==============================] - 1s 182us/sample - loss: 0.1294 - accuracy: 0.9623 - val_loss: 0.6950 - val_accuracy: 0.8470
Epoch 58/100
8000/8000 [==============================] - 1s 183us/sample - loss: 0.1262 - accuracy: 0.9629 - val_loss: 0.7071 - val_accuracy: 0.8466
Epoch 59/100
8000/8000 [==============================] - 1s 184us/sample - loss: 0.1239 - accuracy: 0.9638 - val_loss: 0.7060 - val_accuracy: 0.8472
Epoch 60/100
8000/8000 [==============================] - 2s 189us/sample - loss: 0.1212 - accuracy: 0.9646 - val_loss: 0.7221 - val_accuracy: 0.8462
Epoch 61/100
8000/8000 [==============================] - 1s 186us/sample - loss: 0.1185 - accuracy: 0.9654 - val

In [12]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 71)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 85)]   0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 256), (None, 335872      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  350208      input_2[0][0]                    
                                                                 lstm[0][1]                   

In [13]:
#loadedmodel = tensorflow.keras.models.load_model("s2s")
#loadedmodel.summary()

In [14]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
#loadedmodel = tensorflow.keras.models.load_model("s2s")
#loadedmodel.summary()

#encoder_inputs = loadedmodel.input[0]  # input_1
#encoder_outputs, state_h_enc, state_c_enc = loadedmodel.layers[2].output  # lstm_1
#encoder_states = [state_h_enc, state_c_enc]
encoder_model = tensorflow.keras.Model(encoder_inputs, encoder_states)

#decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = tensorflow.keras.Input(shape=(latent_dim,), name="input_3")
decoder_state_input_c = tensorflow.keras.Input(shape=(latent_dim,), name="input_4")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = tensorflow.keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [15]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Geh.

-
Input sentence: Hi.
Decoded sentence: Hallo!

-
Input sentence: Hi.
Decoded sentence: Hallo!

-
Input sentence: Run!
Decoded sentence: Lauf!

-
Input sentence: Run.
Decoded sentence: Lauf!

-
Input sentence: Wow!
Decoded sentence: Donnerwetter!

-
Input sentence: Wow!
Decoded sentence: Donnerwetter!

-
Input sentence: Fire!
Decoded sentence: Feuer!

-
Input sentence: Help!
Decoded sentence: Zu Hülf!

-
Input sentence: Help!
Decoded sentence: Zu Hülf!

-
Input sentence: Stop!
Decoded sentence: Anhalten!

-
Input sentence: Stop!
Decoded sentence: Anhalten!

-
Input sentence: Wait!
Decoded sentence: Warte!

-
Input sentence: Wait.
Decoded sentence: Warte.

-
Input sentence: Begin.
Decoded sentence: Fangen Sie an.

-
Input sentence: Go on.
Decoded sentence: Mach weiter.

-
Input sentence: Hello!
Decoded sentence: Sers!

-
Input sentence: Hello!
Decoded sentence: Sers!

-
Input sentence: Hurry!
Decoded sentence: Beeil dich!

-
Input sentence: 